In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso, Ridge, ElasticNet
from sklearn import metrics
from sklearn.preprocessing import StandardScaler

In [ ]:
from google.colab import files

# Upload a single file
uploaded = files.upload()

# To upload multiple files
# uploaded = files.upload()

# Display the uploaded file names
for filename in uploaded.keys():
    print(f'File uploaded: {filename}')

Saving imputed_training_data.csv to imputed_training_data.csv
File uploaded: imputed_training_data.csv


In [ ]:
df = pd.read_csv("imputed_training_data.csv")
df = df[:2254]
df = df.drop(df.columns[[0,1]], axis=1)
df = df.drop(columns="id")
df = df.dropna()

In [ ]:
df.head()

,comma,period,dash,qm,ep,sc,colon,revision_ratio,total_actions,one_min_pause_count,...,mean_action_time_25_30,std_action_time_25_30,skew_action_time_25_30,max_inter_action_time_25_30,min_inter_action_time_25_30,median_inter_action_time_25_30,mean_inter_action_time_25_30,std_inter_action_time_25_30,skew_inter_action_time_25_30,score
0,0.400000,0.560000,0.000000,0.000000,0.0,0.04,0.0,0.171817,2557.0,2.0,...,65.513514,42.091955,0.299878,45.000000,154169.000000,295.50000,3409.391892,17976.440849,8.310227,3.5
1,0.486486,0.324324,0.108108,0.081081,0.0,0.00,0.0,0.118289,2454.0,3.0,...,65.924528,71.300798,4.362933,1.000000,114611.000000,36.00000,2630.726415,13556.384903,6.555030,3.5
2,0.500000,0.475000,0.025000,0.000000,0.0,0.00,0.0,0.152399,4136.0,4.0,...,106.787724,100.892573,7.943101,2.000000,74218.000000,100.00000,584.396419,4680.999225,13.580859,6.0
3,0.187500,0.812500,0.000000,0.000000,0.0,0.00,0.0,0.103780,1556.0,2.0,...,148.751892,183.272933,1.569110,185.181756,69010.712523,6141.72364,12049.176605,17253.751125,4.322375,2.0
4,0.589744,0.358974,0.000000,0.051282,0.0,0.00,0.0,0.210248,2531.0,3.0,...,121.394161,36.492013,-0.060196,18.000000,37054.000000,221.00000,829.014599,3764.831099,8.236028,4.0


In [ ]:
cnt=0
for i in df['score']:
  if i<=5 and i>=3:
    cnt+=1
print(cnt/len(df))

0.7710736468500443


In [ ]:
corr = df.corr()

In [ ]:
cnt=0
for i in corr['score']:
  if i<=0.2 and i>=-0.2:
    cnt+=1
print(cnt/len(corr))

0.7430830039525692




> ## Feature Selection Methods



In [ ]:
# Whole Features

X = df.drop(df.columns[-1], axis=1).values
y = df.iloc[:, -1].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Filter Method

from sklearn.model_selection import train_test_split

def filter_method(df,corr_level_score=0.2,corr_level_feature=0.9):
  corr = df.corr()
  features = []
  for i in range(len(corr.columns)):
    feature = corr.columns[i]
    if abs(corr['score'][feature])<corr_level_score: continue
    else:
      for j in range(i+1,len(corr.columns)):
        alt_feature = corr.columns[j]
        if abs(corr[feature][alt_feature])>corr_level_feature and abs(corr['score'][alt_feature])>abs(corr['score'][feature]):
          feature = alt_feature
    if feature not in features: features.append(feature)
  additional_features = ['action_occured_0_5', 'action_occured_5_10', 'action_occured_10_15','action_occured_15_20','action_occured_20_25','action_occured_25_30']

  df = df.dropna()
  X = df.drop('score', axis=1)
  y = df['score']
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
  return X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test = filter_method(df,corr_level_score=0.2,corr_level_feature=0.9)

In [ ]:
#PCA

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pandas as pd

def PCA_method(df,n_components):
  df = df.dropna()
  y = df['score']
  X = df.drop('score', axis=1)
  scaler = StandardScaler()
  X_scaled = scaler.fit_transform(X)

  pca = PCA(n_components=n_components)

  X_pca = pca.fit_transform(X_scaled)

  X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=42)

  return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = PCA_method(df,25)



> ## SVM




In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn import metrics
import numpy as np

# Split the data into training and testing sets
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define the hyperparameters to tune
hyperparameters = {'C': np.logspace(-4, 4, 10, 20), 'kernel': ['rbf']}

# Create a SVR object
svr = SVR()

# Create a GridSearchCV object
clf = GridSearchCV(svr, hyperparameters, cv=5, verbose=0)

# Fit the model to the training data
clf.fit(X_train, y_train)

# Make predictions on the testing set
y_pred = clf.predict(X_test)


# Create a DataFrame
df2 = pd.DataFrame({'Predicted Values': y_pred, 'Target Values': y_test})

# Print the DataFrame
print(df2)

# Calculate and print the Mean Squared Error of the model on the testing set
mse = metrics.mean_squared_error(y_test, y_pred)
print('Mean Squared Error:', mse)

# Calculate and print the Root Mean Squared Error of the model on the testing set
rmse = np.sqrt(mse)
print('Root Mean Squared Error:', rmse)

# Print the best parameters
print('Best parameters:', clf.best_params_)


      Predicted Values  Target Values
2063          3.286959            3.0
1263          3.931010            4.0
2127          3.464509            3.5
599           3.256469            3.0
464           4.558876            3.5
...                ...            ...
1880          3.242204            2.5
427           4.334780            4.5
374           3.172884            3.0
105           3.469115            3.0
1966          3.305714            2.5

[451 rows x 2 columns]
Mean Squared Error: 0.46225210910768294
Root Mean Squared Error: 0.679891248000504
Best parameters: {'C': 0.3593813663804626, 'kernel': 'rbf'}


In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Create a decision tree regressor instance
regressor = DecisionTreeRegressor()

# Fit the regressor on the training data
regressor.fit(X_train, y_train)

# To make predictions on the test set
predictions = regressor.predict(X_test)

# Evaluate the model using metrics like Mean Squared Error (MSE) and R-squared (R2)
mse = mean_squared_error(y_test, predictions)

rmse=np.sqrt(mse)

# You can print or return the evaluation metrics as needed
print(f"Mean Squared Error: {rmse}")


Mean Squared Error: 0.9764408876605607


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

rf = RandomForestRegressor(n_estimators=200, random_state=27)

rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)

mse = mean_squared_error(y_test, y_pred)

#compare = pd.concat([y_test.reset_index(drop=True), pd.Series(y_pred)], axis=1)
#compare.columns = ['Actual', 'Predicted']

print(f"Mean Squared Error: {mse}")

Mean Squared Error: 0.47275443458980043


In [ ]:
def evaluate_regressor(reg, X_test, y_test):
  # Compute the mse, rmse on the test set
  ### START CODE ###
  y_pred = reg.predict(X_test)
  mse = mean_squared_error(y_test, y_pred)
  rmse = mean_squared_error(y_test, y_pred, squared=False)
  ### END CODE ###
  print("MSE: {}, RMSE: {}".format(mse,rmse))
def train_and_evaluate_regressor(reg, X_train, y_train, X_test, y_test):
  t0 = time()
  # Fit your classifier on the training set
  ### START CODE ###
  reg.fit(X_train, y_train)
  ### END CODE ###
  print("training time", round(time()-t0, 3), "s")

  t0 = time()
  y_pred = reg.predict(X_test)
  print("predict time", round(time()-t0, 3), "s")

  # Compute the mse, rmse on the test set
  ### START CODE ###
  mse = mean_squared_error(y_test, y_pred)
  rmse = mean_squared_error(y_test, y_pred, squared=False)
  ### END CODE ###

  print("MSE: {}, RMSE: {}".format(mse,rmse))
  return reg
def fit_regressor(reg, X_train, y_train):
  # Fit the regressor on the training set
  ### START CODE ###
  reg.fit(X_train, y_train)
  ### END CODE ###
  return reg

In [ ]:
from sklearn import preprocessing
from sklearn.neural_network import MLPRegressor

X_train_scaled = preprocessing.scale(X_train, axis=0, with_mean=True, with_std=True, copy=True)
X_test_scaled = preprocessing.scale(X_test, axis=0, with_mean=True, with_std=True, copy=True)

print("Neural Network (Default)")
nn_reg = MLPRegressor()
nn_reg = fit_regressor(nn_reg, X_train_scaled, y_train)
evaluate_regressor(nn_reg, X_test_scaled, y_test)


Neural Network (Default)
MSE: 0.9212561778926217, RMSE: 0.9598209092808




> ## Lasso/Ridge/ElasticNet



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso, Ridge, ElasticNet
from sklearn import metrics
from sklearn.preprocessing import StandardScaler

In [ ]:
# Define the hyperparameters to tune
hyperparameters = {'alpha': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100]}


# Create Lasso, Ridge, and ElasticNet objects
lasso = Lasso()
ridge = Ridge()
elastic_net = ElasticNet()
# List of number of folds
folds = [10]
# List of test sizes
test_sizes = [0.2]
# Create a DataFrame to store the MSE for each regression
mse_df = pd.DataFrame(columns=['Test Size', 'K-Fold', 'Lasso MSE', 'Ridge MSE', 'ElasticNet MSE'])

for test_size in test_sizes:
    print(f'Performing train-test split with test size {test_size}')

    for fold in folds:
        print(f'Performing cross-validation with {fold} folds')

        # Create GridSearchCV objects
        lasso_regressor = GridSearchCV(lasso, hyperparameters, scoring='neg_mean_squared_error', cv=KFold(n_splits=5))
        ridge_regressor = GridSearchCV(ridge, hyperparameters, scoring='neg_mean_squared_error', cv=KFold(n_splits=5))
        elastic_net_regressor = GridSearchCV(elastic_net, hyperparameters, scoring='neg_mean_squared_error', cv=KFold(n_splits=5))

        # Fit the models to the training data
        lasso.fit(X_train, y_train)
        ridge.fit(X_train, y_train)
        elastic_net.fit(X_train, y_train)

        # Make predictions on the testing set
        y_pred_lasso = lasso.predict(X_test)
        y_pred_ridge = ridge.predict(X_test)
        y_pred_elastic_net = elastic_net.predict(X_test)

        # Print the Mean Squared Error of the models on the testing set
        print('Lasso Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred_lasso))
        print('Ridge Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred_ridge))
        print('ElasticNet Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred_elastic_net))

        # Calculate the MSE for each regression
        lasso_mse = metrics.mean_squared_error(y_test, y_pred_lasso)
        ridge_mse = metrics.mean_squared_error(y_test, y_pred_ridge)
        elastic_net_mse = metrics.mean_squared_error(y_test, y_pred_elastic_net)

# Append the MSE to the DataFrame
mse_df = mse_df.append({'Test Size': test_size, 'K-Fold': fold, 'Lasso MSE': lasso_mse, 'Ridge MSE': ridge_mse, 'ElasticNet MSE': elastic_net_mse}, ignore_index=True)

# Print the DataFrame
print(mse_df)


Performing train-test split with test size 0.2
Performing cross-validation with 10 folds


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.973e+02, tolerance: 1.905e-01
  model = cd_fast.enet_coordinate_descent(


Lasso Mean Squared Error: 0.7567100782730101
Ridge Mean Squared Error: 2.2690882905680088
ElasticNet Mean Squared Error: 1.3226059959548915
   Test Size  K-Fold  Lasso MSE  Ridge MSE  ElasticNet MSE
0        0.2    10.0    0.75671   2.269088        1.322606


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.971e+02, tolerance: 1.905e-01
  model = cd_fast.enet_coordinate_descent(
<ipython-input-52-e6edee40c0db>:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mse_df = mse_df.append({'Test Size': test_size, 'K-Fold': fold, 'Lasso MSE': lasso_mse, 'Ridge MSE': ridge_mse, 'ElasticNet MSE': elastic_net_mse}, ignore_index=True)
